# 数据准备

In [4]:
import os
import shutil
from glob2 import glob
import numpy as np


In [20]:
fitered_set = set(np.load('cleaned_file_list.npy'))

In [21]:
len(fitered_set)

29155

In [ ]:
if 'data' not in os.listdir():
    os.mkdir('data/')
    os.mkdir('data/train')
    os.mkdir('data/train/cat')    
    os.mkdir('data/train/dog')    
    os.mkdir('data/test')    
    os.mkdir('data/test/test')  
    
for path in glob('train/*'):
    filename = path.split('/')[-1]
    if filename in fitered_set:
        if 'cat' in filename:
            shutil.move(path, 'data/train/cat/')
        else:
            shutil.move(path, 'data/train/dog/')

for path in glob('test/*.jpg'):
    shutil.move(path, 'data/test/test/')

    
cap = re.compile('[A-Z]')

for path in glob('images/*'):
    if len(cap.findall(path)):
        shutil.move(path, 'data/train/cat/')
    else:
        shutil.move(path, 'data/train/dog/')
    


# 特征提取

In [22]:
from keras.applications import *
from keras.preprocessing import image

In [ ]:
# 加载预训练模型
model_xception = Xception(weights='imagenet',include_top=False, pooling='avg')
model_resnet50 = ResNet50(weights='imagenet',include_top=False, pooling='avg')
model_vgg16 = VGG16(weights='imagenet',include_top=False, pooling='avg')
model_inceptionv3 = InceptionV3(weights='imagenet', include_top=False, pooling='avg')
model_incresnet = InceptionResNetV2(weights='imagenet', include_top=False, pooling='avg')
model_dense = DenseNet201(weights='imagenet', include_top=False, pooling='avg')

In [ ]:
# 通用图片特征提取器
def get_bottleneck_feature(pretrain_model,model_name, datagen, img_size):
    train_generator = datagen.flow_from_directory(
            'data/train',
            target_size=img_size,
            batch_size=100,
            shuffle=False,
            class_mode='binary')
    
    test_generator = datagen.flow_from_directory(
            'data/test',
            target_size=img_size,
            batch_size=50,
            shuffle=False,
            class_mode=None)

    model = Model(pretrain_model.input, GlobalAveragePooling2D()(pretrain_model.output))
    
    train_data = model.predict_generator(train_generator, 292)
    test_data = model.predict_generator(test_generator, 250)
    with h5py.File("gap_{}.h5".format(model_name)) as h:
        h.create_dataset('train',data=train_data)
        h.create_dataset('label_train',data=train_generator.classes)
        h.create_dataset('test',data=test_data)

In [ ]:
# 提取图片特征
get_bottleneck_feature(model_xception,'xception',image.ImageDataGenerator(preprocessing_function=xception.preprocess_input),(299,299))
get_bottleneck_feature(model_resnet50,'resnet50',image.ImageDataGenerator(preprocessing_function=resnet50.preprocess_input),(224,224))
get_bottleneck_feature(model_vgg16,'vgg16',image.ImageDataGenerator(preprocessing_function=vgg16.preprocess_input),(224,224))
get_bottleneck_feature(model_inceptionv3,'inception',image.ImageDataGenerator(preprocessing_function=inception_v3.preprocess_input),(299,299))
get_bottleneck_feature(model_incresnet,'inception_resnet',image.ImageDataGenerator(preprocessing_function=inception_resnet_v2.preprocess_input),(299,299))
get_bottleneck_feature(model_dense,'densenet',image.ImageDataGenerator(preprocessing_function=densenet.preprocess_input),(224,224))